In [7]:
# Importing Libraries
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

In [9]:
# Connect MySQL database
host = 'localhost'
port = '3306'
username = 'root'
password = 'root'
database_schema = 'text_to_sql'

mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info=2)


In [11]:
db = SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info=1)

db.get_table_info()

'\nCREATE TABLE `2017_budgets` (\n\t`Product Name` TEXT, \n\t`2017 Budgets` DOUBLE\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from 2017_budgets table:\nProduct Name\t2017 Budgets\nProduct 1\t3016489.2089999998\n*/\n\n\nCREATE TABLE customers (\n\t`Customer Index` INTEGER, \n\t`Customer Names` TEXT\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from customers table:\nCustomer Index\tCustomer Names\n1\tGeiss Company\n*/\n\n\nCREATE TABLE products (\n\t`Index` INTEGER, \n\t`Product Name` TEXT\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from products table:\nIndex\tProduct Name\n1\tProduct 1\n*/\n\n\nCREATE TABLE regions (\n\tid INTEGER, \n\tname TEXT, \n\tcounty TEXT, \n\tstate_code TEXT, \n\tstate TEXT, \n\ttype TEXT, \n\tlatitude DOUBLE, \n\tlongitude DOUBLE, \n\tarea_code INTEGER, \n\tpopulation INTEGER, \n\thouseholds INTEGER, \n\tmedian_income INTEGER, \n\tland_area INTEGER, \

In [13]:
# Create the LLM Prompt Template
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
Remember : Only provide me the sql query dont include anything else. Provide me sql query in a single line dont add line breaks
Table Schema: {schema}
Question: {question}
SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [15]:
# get the schema of the database
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [19]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = 'API_KEY_HERE')

In [21]:
# Create the SQL query chain using the LLM and the prompt template

sql_chain = (
    RunnablePassthrough.assign(schema=lambda _:get_schema(db)) 
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [23]:
# test the SQL query chain with a sample question

resp = sql_chain.invoke({"question": "What is the total 'Line Total' for Geiss Company"})
print(resp)

ValueError: Your location is not supported by google-generativeai at the moment. Try to use ChatVertexAI LLM from langchain_google_vertexai.